<a href="https://colab.research.google.com/github/Xjkslhstxn/BT4222/blob/main/1.8%20Ncf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Collaborative Filtering And MLP


### Setting up the environment (~4mins)


In [ ]:
# This is only necessary for colab since it only supports python 3.10, but the library we are using only supports <= 3.9.
# Comment this section if you are running it on your local machine

!sudo rm -rf /usr/local/lib/python3.8/dist-packages/OpenSSL
!sudo rm -rf /usr/local/lib/python3.8/dist-packages/pyOpenSSL-22.1.0.dist-info/

!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.5.2-0-Linux-x86_64.sh

!bash ./Miniconda3-py39_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
!pip3 install pyOpenSSL==22.0.0

# Installing the recommenders library.
# Ensure that you have python version <=3.9 when installing this.
!pip install recommenders[examples]

--2024-04-08 13:18:50--  https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93409434 (89M) [application/x-sh]
Saving to: ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.2’

Miniconda3-py39_23. 100%[===================>]  89.08M   132MB/s    in 0.7s    

2024-04-08 13:18:51 (132 MB/s) - ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.2’ saved [93409434/93409434]

PREFIX=/usr/local
Unpacking payload ...
                                                                            
Installing base environment...



Preparing transaction: - done
Executing transaction: | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in

### Importing Libraries

In [ ]:
import sys
import os
import shutil

# Pandas and Numpy is used for efficient handling of arrays.
import pandas as pd
import numpy as np


from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_chrono_split

# importing the dataset
from recommenders.datasets import movielens
from recommenders.models.ncf.dataset import Dataset as NCFDataset

# Importing the NCF model class from the recommenders library
from recommenders.models.ncf.ncf_singlenode import NCF

# importing the evaluation metrics
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Pandas version: 2.0.3


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/bt4222_notebook/

[Errno 2] No such file or directory: 'drive/MyDrive/bt4222_notebook/'
/content/drive/MyDrive/bt4222_notebook


In [ ]:
data_df = pd.read_csv('UserWithLabelAndInput_0.csv')

In [ ]:
data_df.head(3)

,UserId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,top1_id,top1_score,top2_id,top2_score,bottom1_id,bottom1_score,bottom2_id,bottom2_score,Input_Merged,embeddings
0,41504,I agree,I am a developer by profession,25-34 years old,"Not employed, but looking for work",NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Online Courses or Certification;Other online r...,Formal documentation provided by the owner of ...,...,10740,0.638643,3778,0.565087,1873,-0.007485,16138,0.016606,C#;Kotlin;Python;SQL Dynamodb;PostgreSQL;SQLit...,[-1.78260654e-02 -8.13032761e-02 -5.13622612e-...
1,21778,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,Some college/university study without earning ...,Colleague;Online Courses or Certification;Othe...,Formal documentation provided by the owner of ...,...,10497,0.635020,469,0.529684,15923,-0.021622,12372,-0.009894,PHP MySQL;PostgreSQL;Redis;SQLite Amazon Web S...,[-6.85331179e-03 -6.82291240e-02 -4.92867678e-...
2,47054,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Other online resources (e.g., videos, blogs, f...",Formal documentation provided by the owner of ...,...,673,0.631075,10740,0.589067,9414,0.025441,15923,0.028745,Java;Python;SQL BigQuery;PostgreSQL;Redis Goog...,[-5.27353063e-02 -4.61720899e-02 8.59866943e-...


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 97 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   UserId                               2000 non-null   int64  
 1   Q120                                 2000 non-null   object 
 2   MainBranch                           2000 non-null   object 
 3   Age                                  2000 non-null   object 
 4   Employment                           1999 non-null   object 
 5   RemoteWork                           1656 non-null   object 
 6   CodingActivities                     1655 non-null   object 
 7   EdLevel                              2000 non-null   object 
 8   LearnCode                            2000 non-null   object 
 9   LearnCodeOnline                      2000 non-null   object 
 10  LearnCodeCoursesCert                 880 non-null    object 
 11  YearsCode                     

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# reshaped_df = pd.DataFrame()
reshaped_list = []
job_types = ['top1', 'top2', 'bottom1', 'bottom2']

for idx, row in data_df.iterrows():
    for job_type in job_types:
        job_id = row[f'{job_type}_id']
        score = row[f'{job_type}_score']
        reshaped_list.append({
            'userID': row['UserId'],
            'itemID': job_id,
            'embeddings': row['embeddings'],
            'rating': score
        })

reshaped_df = pd.DataFrame(reshaped_list)

reshaped_df['userID'] = reshaped_df['userID'].astype(int)
reshaped_df['itemID'] = reshaped_df['itemID'].astype(int)
reshaped_df['rating'] = reshaped_df['rating'].astype(float)

In [ ]:
print(reshaped_df.head(3))

   userID  itemID                                         embeddings    rating
0   41504   10740  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...  0.638643
1   41504    3778  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...  0.565087
2   41504    1873  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-... -0.007485


In [ ]:
X = reshaped_df[['userID', 'itemID']]  # Features
y = reshaped_df['rating']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_df = pd.DataFrame(X_train)
train_df['rating'] = y_train
train_df = train_df.sort_values(by='userID')

test_df = pd.DataFrame(X_test)
test_df['rating'] = y_test
filtered_test_df = test_df[
    (test_df['userID'].isin(train_df['userID'].unique())) &
    (test_df['itemID'].isin(train_df['itemID'].unique()))
]

In [ ]:
filtered_test_df.info()
filtered_test_df = filtered_test_df.sort_values(by='userID')

<class 'pandas.core.frame.DataFrame'>
Index: 1541 entries, 2215 to 6832
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   userID  1541 non-null   int64  
 1   itemID  1541 non-null   int64  
 2   rating  1541 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 48.2 KB


In [ ]:
leave_one_out_test = filtered_test_df.groupby('userID').head(1)
rest_of_interactions = filtered_test_df.groupby('userID').apply(lambda x: x.iloc[1:]).reset_index(drop=True)
train_df = pd.concat([train_df, rest_of_interactions])
train_df = train_df.sort_values(by='userID')


In [ ]:
train_file = "./ncf_train_file_0.csv.csv"
leave_one_out_test_file = "./ncf_test_file_0.csv"

train_df.to_csv(train_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file , index=False)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6799 entries, 4687 to 6975
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   userID  6799 non-null   int64  
 1   itemID  6799 non-null   int64  
 2   rating  6799 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 212.5 KB


In [ ]:
leave_one_out_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1142 entries, 4686 to 6972
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   userID  1142 non-null   int64  
 1   itemID  1142 non-null   int64  
 2   rating  1142 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 35.7 KB


In [ ]:
EPOCHS = 25
BATCH_SIZE = 256
SEED = 42

data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

100%|██████████| 1142/1142 [00:11<00:00, 95.41it/s]


In [ ]:
model_1 = NCF(
    n_users=50000,
    n_items=22000,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16, 8, 4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [ ]:
with Timer() as train_time:
    model_1.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 249.85612845700052 seconds for training.


In [ ]:
# predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
#                for (_, row) in filtered_test_df.iterrows()]
global_average = train_df['rating'].mean()
predictions = []
for (_, row) in reshaped_df.iterrows():
    user_id = row.userID
    item_id = row.itemID
    # Check if both user and item are known from the training phase
    if user_id in model_1.user2id and item_id in model_1.item2id:
        prediction = model_1.predict(user_id, item_id)
    else:
        prediction = global_average
    predictions.append([user_id, item_id, prediction])


predictions_df = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])

In [ ]:
result_df = pd.merge(reshaped_df, predictions_df, on=['userID', 'itemID'], how='left')

In [ ]:
print(result_df.head(10))

   userID  itemID                                         embeddings  \
0   41504   10740  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
1   41504    3778  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
2   41504    1873  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
3   41504   16138  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
4   21778   10497  [-6.85331179e-03 -6.82291240e-02 -4.92867678e-...   
5   21778     469  [-6.85331179e-03 -6.82291240e-02 -4.92867678e-...   
6   21778   15923  [-6.85331179e-03 -6.82291240e-02 -4.92867678e-...   
7   21778   12372  [-6.85331179e-03 -6.82291240e-02 -4.92867678e-...   
8   47054     673  [-5.27353063e-02 -4.61720899e-02  8.59866943e-...   
9   47054   10740  [-5.27353063e-02 -4.61720899e-02  8.59866943e-...   

     rating  prediction  
0  0.638643    0.796670  
1  0.565087    0.793722  
2 -0.007485    0.033334  
3  0.016606    0.768523  
4  0.635020    0.980841  
5  0.529684    0.852716  
6 -0.021622    0.000216  

In [ ]:
jobs_df = pd.read_csv('JobsWithInput.csv')


In [ ]:
result_df['itemID'] = result_df['itemID'].astype(int)
jobs_df['JobId'] = jobs_df['JobId'].astype(int)
jobs_df['jobs_embeddings'] = jobs_df['embeddings']
merged_df = pd.merge(result_df, jobs_df[['JobId', 'jobs_embeddings']], left_on='itemID', right_on='JobId', how='left')
print(merged_df)

      userID  itemID                                         embeddings  \
0      41504   10740  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
1      41504    3778  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
2      41504    1873  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
3      41504   16138  [-1.78260654e-02 -8.13032761e-02 -5.13622612e-...   
4      21778   10497  [-6.85331179e-03 -6.82291240e-02 -4.92867678e-...   
...      ...     ...                                                ...   
8895   35569    2970  [-4.69129644e-02 -9.12079737e-02  1.03325062e-...   
8896   41470   17582  [-6.39124494e-03 -7.74837658e-02  7.68335611e-...   
8897   41470   18070  [-6.39124494e-03 -7.74837658e-02  7.68335611e-...   
8898   41470   17827  [-6.39124494e-03 -7.74837658e-02  7.68335611e-...   
8899   41470   15440  [-6.39124494e-03 -7.74837658e-02  7.68335611e-...   

        rating  prediction  JobId  \
0     0.638643    0.796670  10740   
1     0.565087    0.79372

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

def convert_str_to_array(embed_str):
    embed_str = embed_str[1:-1]
    str_nums = embed_str.strip().split()
    return np.array([float(num) for num in str_nums if num], dtype=float)

merged_df['embeddings'] = merged_df['embeddings'].apply(convert_str_to_array)
merged_df['jobs_embeddings'] = merged_df['jobs_embeddings'].apply(convert_str_to_array)

embeddings_stack = np.vstack(merged_df['embeddings'].values)
jobs_embeddings_stack = np.vstack(merged_df['jobs_embeddings'].values)
predictions = merged_df['prediction'].values[:, None]

X = np.hstack([embeddings_stack, jobs_embeddings_stack, predictions])
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(result_df['rating'].values, dtype=torch.float)

In [ ]:
print(X)

tensor([[-1.7826e-02, -8.1303e-02, -5.1362e-02,  ..., -3.2662e-03,
          7.9262e-02,  7.9667e-01],
        [-1.7826e-02, -8.1303e-02, -5.1362e-02,  ..., -3.2662e-03,
          7.9262e-02,  7.9372e-01],
        [-1.7826e-02, -8.1303e-02, -5.1362e-02,  ..., -3.2662e-03,
          7.9262e-02,  3.3334e-02],
        ...,
        [-6.3912e-03, -7.7484e-02,  7.6834e-02,  ..., -6.9407e-02,
         -1.9994e-02,  4.8797e-01],
        [-6.3912e-03, -7.7484e-02,  7.6834e-02,  ..., -6.9407e-02,
         -1.9994e-02,  4.8711e-04],
        [-6.3912e-03, -7.7484e-02,  7.6834e-02,  ..., -6.9407e-02,
         -1.9994e-02,  7.3725e-02]])


In [ ]:
print(y)

tensor([ 0.6386,  0.5651, -0.0075,  ...,  0.1753, -0.1447, -0.1443])


In [ ]:
print(len(X[0]))

769


In [ ]:
class RatingDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
dataset = RatingDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, output_size)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
dropout_rate = 0.25
model_2 = MLP(input_size=769, output_size=1)

In [ ]:
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.001)
criterion = nn.MSELoss()

epochs = 10
for epoch in range(epochs):
    model_2.train()
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model_2(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.00036235240986570716
Epoch 2, Loss: 0.0016595158958807588
Epoch 3, Loss: 0.00163693493232131
Epoch 4, Loss: 0.0012191229034215212
Epoch 5, Loss: 0.0074235606007277966
Epoch 6, Loss: 0.001969831297174096
Epoch 7, Loss: 0.019783519208431244
Epoch 8, Loss: 0.011574897915124893
Epoch 9, Loss: 0.0017250437522307038
Epoch 10, Loss: 0.0044236863031983376


In [ ]:
testinginput_df = pd.read_csv('embedded_users_IdEncoded.csv')
print(testinginput_df.info())

# print(run(1,1,model_1, model_2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 87 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   UserId                               50000 non-null  int64  
 1   Q120                                 50000 non-null  object 
 2   MainBranch                           50000 non-null  object 
 3   Age                                  50000 non-null  object 
 4   Employment                           49967 non-null  object 
 5   RemoteWork                           41751 non-null  object 
 6   CodingActivities                     41725 non-null  object 
 7   EdLevel                              50000 non-null  object 
 8   LearnCode                            50000 non-null  object 
 9   LearnCodeOnline                      50000 non-null  object 
 10  LearnCodeCoursesCert                 22108 non-null  object 
 11  YearsCode                   

In [ ]:
testinginput_df.head(3)
print(testinginput_df['NEWCollabToolsHaveWorkedWith'][1])

PyCharm;Sublime Text


In [ ]:


!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model_3 = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
Tech_extract = [
    'LanguageHaveWorkedWith',
    'DatabaseHaveWorkedWith',
    'PlatformHaveWorkedWith',
    'WebframeHaveWorkedWith',
    'MiscTechHaveWorkedWith',
    'ToolsTechHaveWorkedWith',
    'NEWCollabToolsHaveWorkedWith',
    'OpSysPersonal use',
    'OfficeStackAsyncHaveWorkedWith',
    'OfficeStackSyncHaveWorkedWith',
    'AISearchHaveWorkedWith',
    'AIDevHaveWorkedWith'
]
extracted_df = testinginput_df[Tech_extract]

row_index = 41504

def concatenate_tech_columns(row):
    filtered_values = [str(value) for value in row if pd.notnull(value)]
    return ' '.join(filtered_values)

In [ ]:
merged_input_for_row = concatenate_tech_columns(extracted_df.iloc[row_index])
print(merged_input_for_row)
specific_row_df = pd.DataFrame({'Input_Merged': [merged_input_for_row]}, index=[row_index])
print(specific_row_df)

C#;Kotlin;Python;SQL Dynamodb;PostgreSQL;SQLite Amazon Web Services (AWS) Android Studio;PyCharm;Visual Studio Code MacOS;Windows;Windows Subsystem for Linux (WSL) GitHub Discussions Discord;Zoom
                                            Input_Merged
41504  C#;Kotlin;Python;SQL Dynamodb;PostgreSQL;SQLit...


In [ ]:
embeddings = model_3.encode(specific_row_df['Input_Merged'].tolist())
print(len(embeddings[0]))

def convert_str_to_array(embed_str):
    embed_str = embed_str[1:-1]
    str_nums = embed_str.strip().split()
    return np.array([float(num) for num in str_nums if num], dtype=float)

def run(userID, itemID, embeddings, ncf_model, mlp_model):
  global_average = train_df['rating'].mean()
  mlp_model.eval()
  job_embed = jobs_df['jobs_embeddings'][itemID]
  job_embed = convert_str_to_array(job_embed)

  if userID in ncf_model.user2id and itemID in ncf_model.item2id:
        prediction = ncf_model.predict(userID, itemID)
  else:
        prediction = global_average
  combined_input = np.hstack([embeddings, job_embed, prediction])
  X = torch.tensor(combined_input, dtype=torch.float)
  final_prediction = mlp_model(X).detach().numpy()[0]

  return final_prediction

print(run(41504,10740,embeddings[0],model_1, model_2))
print(run(41504,3115,embeddings[0],model_1, model_2))
print(run(41504,88,embeddings[0],model_1, model_2))
print(run(41504,13035,embeddings[0],model_1, model_2))

384
0.44695848
-0.07301435
-0.065692574
-0.025760263


In [ ]:

eval_jobs = pd.read_csv('1k-jobs-eval.csv')
print(eval_jobs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   JobId                     1000 non-null   int64 
 1   advertiserurl             1000 non-null   object
 2   company                   996 non-null    object
 3   employmenttype_jobstatus  994 non-null    object
 4   jobdescription            1000 non-null   object
 5   joblocation_address       1000 non-null   object
 6   jobtitle                  1000 non-null   object
 7   postdate                  1000 non-null   object
 8   shift                     989 non-null    object
 9   site_name                 144 non-null    object
 10  skills                    991 non-null    object
 11  Job_Features_Merged       1000 non-null   object
 12  processed_text            1000 non-null   object
 13  job_embeddings            1000 non-null   object
dtypes: int64(1), object(13)
m

In [ ]:

def apply_run(userID, embeddings, row, model_1, model_2):
    itemID = row['JobId']
    prediction = run(userID, itemID, embeddings, model_1, model_2)
    return prediction

eval_jobs['prediction'] = eval_jobs.apply(lambda row: apply_run(41504, embeddings[0], row, model_1, model_2), axis=1)


In [ ]:
final_df = eval_jobs[['JobId', 'prediction']]
print(final_df.head(30))

    JobId  prediction
0   13035   -0.025760
1    3115   -0.073014
2    8732   -0.033482
3    7591   -0.060701
4     221   -0.079046
5   17295    0.000416
6    5857    0.103001
7   12608   -0.074984
8    3463   -0.040303
9    6478   -0.073173
10   3350    0.257050
11  15982   -0.066899
12  19275   -0.062802
13  18089   -0.035133
14  18753   -0.055205
15  11503   -0.083462
16   4058    0.147775
17   4874   -0.049025
18  19936    0.008795
19  14055    0.319925
20  16852    0.009482
21  18250    0.188128
22  15813    0.058889
23  16414    0.108185
24  19453   -0.077532
25   1489    0.097871
26  11967   -0.089849
27   9561    0.099095
28   1398   -0.038273
29  13829    0.103764


In [ ]:

final_df.to_csv('job_predictions_0.csv', index=False)